In [ ]:
# !pip install ../input/python-datatable/datatable-0.11.0-cp37-cp37m-manylinux2010_x86_64.whl

In [1]:
# import datatable as dt
import pandas as pd
import numpy as np
import random
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import psutil
from sklearn.metrics import roc_auc_score
import pickle
from collections import defaultdict
import json
from time import time
import pickle
from bitarray import bitarray

pd.options.mode.chained_assignment = None
_ = np.seterr(divide='ignore', invalid='ignore')
pd.set_option('display.max_columns', None)

In [2]:
data_types = {
    'row_id': 'int32',
    'timestamp': 'uint64',
    'user_id': 'uint32',
    'content_id': 'uint16',
    'content_type_id': 'int8',
    'task_container_id': 'uint16',
    'prior_question_elapsed_time': 'float32',
    'prior_question_had_explanation': 'boolean'
}

In [3]:
# def jsonKeys2int(x):
#     return defaultdict(int,{int(k):v for k,v in x.items()})

# with open("../input/riiid-attempts-json/attempts.json","r") as f:
#     attempts_dict = json.load(f,object_hook=jsonKeys2int)

# attempts_dict = defaultdict(lambda: defaultdict(int),attempts_dict)

In [4]:
def jsonKeys2int(x):
    return {int(k):v for k,v in x.items()}

with open("../input/riiid-attempts-json/attempts_days.json","r") as f:
    attempts_dict = json.load(f,object_hook=jsonKeys2int)
attempts_dict = defaultdict(dict,attempts_dict)

# attempts_dict = defaultdict(dict,{})

In [5]:
with open('../input/riiid-education/new_questions.pickle', 'rb') as handle:
    quest_dict = pickle.load(handle)
with open('../input/riiid-education/user_dict_pun.pickle', 'rb') as handle:
    user_dict = pickle.load(handle)
with open('../input/riiid-education/model_r.pickle', 'rb') as handle:
    model = pickle.load(handle)
with open('../input/riiid-education/na_fill.pickle', 'rb') as handle:
    na_fill = pickle.load(handle)
with open('../input/riiid-education/train_columns_r.pickle', 'rb') as handle:
    train_columns = pickle.load(handle)
with open('../input/riiid-education/var_timestamp_day_dict.pickle', 'rb') as handle:
    user_dict['ans_var_15_day'] = pickle.load(handle)
with open('../input/riiid-education/lag_timestamp_dict.pickle','rb') as handle:
    user_dict['lag_timestamp'] = pickle.load(handle)
with open('../input/riiid-education/lecture_in_2.pickle','rb') as handle:
    user_dict['lecture_in_2'] = pickle.load(handle)
with open('../input/riiid-education/part_dict.pickle','rb') as handle:
    part_dict = pickle.load(handle)
with open('../input/riiid-education/qn_right_dict.pickle','rb') as handle:
    qn_right_dict = pickle.load(handle)

In [6]:
part_dict['part_cumsum'] = defaultdict(int,part_dict['part_cumsum'])
part_dict['part_cumcount'] = defaultdict(int,part_dict['part_cumcount'])

In [7]:
na_fill['lag_timestamp'] = 25589/(3600*1000)
na_fill['lag_timestamp2'] = 53351/(3600*1000)
na_fill['lag_timestamp3'] = 78124/(3600*1000)
na_fill['lecture_in_2'] = 2
na_fill['ans_var_15_day'] = 3.8677079787433557e-07
na_fill['time_diff'] = 3762/1000
na_fill['prior_question_elapsed_time'] = 21
na_fill['rows'] = 1
na_fill['last_attempt_days'] = np.nan
na_fill['user_time_performance'] = 0
na_fill['user_elapsed_time'] = 210
na_fill['part_bundle_id'] = 0
na_fill['user_punish'] = 0
na_fill['qn_right_prev'] = -1

In [8]:
user_solved_median  = np.median(list(user_dict['user_qn_solved'].values()))
qn_solved_median = np.median(list(quest_dict['qn_count'].values()))

In [9]:
user_cols = ['user_qn_solved', 'user_right_ans', 'user_seen_expln', 'user_elapsed_time', 'user_time_performance','user_punish']
quest_cols = ['qn_part_accuracy', 'qn_tag_accuracy', 'qn_count', 'qn_expln_check', 'qn_median_elapsed_time', 'qn_right_ans','part','rows','part_bundle_id']
feat_cols = ['attempts','lag_timestamp','lag_timestamp2','lag_timestamp3','time_diff','lecture_in_2','ans_var_15_day','qn_right_prev']
part_cols = ['part_cumsum','part_cumcount']

In [10]:
def preprocess(val):
    val = val.astype(data_types)
    val['timestamp'] = (val['timestamp']/1000).astype('float32')
    val['prior_question_elapsed_time'] =  (val['prior_question_elapsed_time']/1000).astype('float32')
  
    val['prior_question_elapsed_time'] = val['prior_question_elapsed_time'].fillna(na_fill['prior_question_elapsed_time'])
    val['prior_question_had_explanation'] = val['prior_question_had_explanation'].fillna(False)
    val['prior_question_had_explanation'] = val['prior_question_had_explanation'].astype('int8')
    val = val.drop(columns=['prior_group_answers_correct','prior_group_responses'])

    length = val.shape[0]

    user_new_cols = { col: np.full(length,np.nan) for col in user_cols}
    quest_new_cols = { col: np.full(length,np.nan) for col in quest_cols}
    feat_new_cols = {col: np.full(length,np.nan) for col in feat_cols}
    part_new_cols = {col: np.full(length,np.nan) for col in part_cols}
    
    for num,row in enumerate(val[['user_id','content_id','prior_question_elapsed_time','content_type_id']].values):
        for col in user_cols:
            user_new_cols[col][num] = user_dict[col].get(row[0],na_fill[col])
        for col in quest_cols:
            quest_new_cols[col][num] = quest_dict[col].get(row[1],na_fill[col])
        
        feat_new_cols['attempts'][num] = attempts_dict[row[0]].get(row[1],np.nan)
        feat_new_cols['lecture_in_2'][num] = user_dict['lecture_in_2'].get(row[0],na_fill['lecture_in_2'])
        
        temp = user_dict['lag_timestamp'].get(row[0],[])
        if len(temp)>=1:
            feat_new_cols['lag_timestamp'][num] = user_dict['lag_timestamp'][row[0]][-1]
            if len(temp)>=2:
                feat_new_cols['lag_timestamp2'][num] = user_dict['lag_timestamp'][row[0]][-2]
                feat_new_cols['time_diff'][num] = (user_dict['lag_timestamp'][row[0]][-1] - user_dict['lag_timestamp'][row[0]][-2]) - row[2]*len(user_dict['content_id'].get(row[0],[]))
                if len(temp)>=3:
                    feat_new_cols['lag_timestamp3'][num] = user_dict['lag_timestamp'][row[0]][-3]
        
        if len(user_dict['ans_var_15_day'].get(row[0],[])) >= 5:
            feat_new_cols['ans_var_15_day'][num] = np.var(user_dict['ans_var_15_day'][row[0]])
        
        if row[3]==0:
            feat_new_cols['qn_right_prev'][num] = qn_right_dict.get(row[0],{row[1]:-1})[int(row[1])]
        
        
        part_new_cols['part_cumsum'][num] = part_dict['part_cumsum'][(row[0],quest_dict['part'].get(row[1],1))]
        part_new_cols['part_cumcount'][num] = part_dict['part_cumcount'][(row[0],quest_dict['part'].get(row[1],1))]
    
    
    val['qn_right_prev'] = feat_new_cols['qn_right_prev']
    val['qn_right_prev'] = val['qn_right_prev'].fillna(-1).astype('int8')
    val['last_attempt_days'] = (val['timestamp']//(3600*24)) - feat_new_cols['attempts']
    val['last_attempt_days'] = val['last_attempt_days'].fillna(-1).astype('int16')
    val.loc[val.last_attempt_days==-1,'qn_right_prev'] = -1
    
    val['lecture_in_2'] = (feat_new_cols['lecture_in_2']<2).astype('uint8')
    val['lag_timestamp'] = (val['timestamp'] - feat_new_cols['lag_timestamp'])/3600
    val['lag_timestamp2'] = (val['timestamp'] - feat_new_cols['lag_timestamp2'])/3600
    val['lag_timestamp3'] = (val['timestamp'] - feat_new_cols['lag_timestamp3'])/3600
    val['ans_var_15_day'] = feat_new_cols['ans_var_15_day']
    val['time_diff'] = feat_new_cols['time_diff']
    
    val[['lecture_in_2','lag_timestamp','lag_timestamp2',
         'lag_timestamp3','ans_var_15_day','time_diff']] = val[['lecture_in_2','lag_timestamp','lag_timestamp2',
                                                                'lag_timestamp3','ans_var_15_day','time_diff']].fillna(na_fill)
    
    val['user_punish'] = user_new_cols['user_punish']/user_new_cols['user_qn_solved']
    val['user_accuracy'] = user_new_cols['user_right_ans']/user_new_cols['user_qn_solved'] * 10000
    val['user_mean_seen_expln'] = user_new_cols['user_seen_expln']/user_new_cols['user_qn_solved'] * 10000
    val['user_mean_elapsed_time'] = user_new_cols['user_elapsed_time']/user_new_cols['user_qn_solved']
    val['user_mean_time_perf'] = user_new_cols['user_time_performance']/user_new_cols['user_qn_solved']
    val['user_qn_solved'] = user_new_cols['user_qn_solved'] / user_solved_median
    
    val['qn_part_accuracy'] = quest_new_cols['qn_part_accuracy']
    val['qn_tag_accuracy'] = quest_new_cols['qn_tag_accuracy']
    val['rows'] = quest_new_cols['rows']
    
    val['qn_right_ans'] = quest_new_cols['qn_right_ans'] / quest_new_cols['qn_count']
    val['qn_median_elapsed_time'] = quest_new_cols['qn_median_elapsed_time'] / quest_new_cols['qn_count']
    val['qn_expln_check'] = quest_new_cols['qn_expln_check'] / quest_new_cols['qn_count']
    val['qn_count'] = quest_new_cols['qn_count'] / qn_solved_median
    val['part'] = quest_new_cols['part']
    val['part_bundle_id'] = quest_new_cols['part_bundle_id']
    
    val['part_cumsum'] = part_new_cols['part_cumsum']
    val['part_cumcount'] = part_new_cols['part_cumcount']
    val['part_accuracy'] = (val['part_cumsum']+5*val['qn_part_accuracy'])/(val['part_cumcount']+5)
    
#     val = val.fillna(na_fill)
    val['hmean'] = 2*((val['qn_right_ans']*val['user_accuracy']/10000)/(val['qn_right_ans'] + val['user_accuracy']/10000))
    
    val[['user_accuracy',
         'user_mean_seen_expln']] = np.rint(val[['user_accuracy',
                                                 'user_mean_seen_expln']]).astype({'user_accuracy':'uint16',
                                                                                   'user_mean_seen_expln':'uint16'})
    return val

In [11]:
def update_from_old(old_data):
    for user_id, timestamp in old_data.loc[old_data.content_type_id==1,['user_id','timestamp']].drop_duplicates(subset='user_id',keep='first').values:
        user_dict['lag_timestamp'][user_id] = user_dict['lag_timestamp'].get(user_id,[]) + [timestamp]
        user_dict['lecture_in_2'][user_id] = 0
    
    
    old_data = old_data.loc[old_data.content_type_id == 0]
    seen_users = set()
    for user_id, content_id, answered_correctly, user_qn_solved, qn_count, timestamp, qn_right_ans in old_data[['user_id','content_id','answered_correctly','user_qn_solved','qn_count','timestamp','qn_right_ans']].values:

        if user_qn_solved == 10/user_solved_median and user_id not in seen_users:
            # This means its a new user
            user_dict['user_qn_solved'][user_id] = 10
            user_dict['user_right_ans'][user_id] = na_fill['user_right_ans']
            user_dict['user_seen_expln'][user_id] = na_fill['user_seen_expln']
            user_dict['user_elapsed_time'][user_id] = na_fill['user_elapsed_time']
            user_dict['user_time_performance'][user_id] = na_fill['user_time_performance']
            user_dict['content_id'][user_id] = set()
            user_dict['lecture_in_2'][user_id] = 2
            user_dict['lag_timestamp'][user_id] = []
            user_dict['ans_var_15_day'][user_id] = [0]
            user_dict['user_punish'][user_id] = 0
            qn_right_dict[user_id] = bitarray(13523,endian='little')
            qn_right_dict[user_id].setall(False)
            
        if user_id not in seen_users:
            user_dict['content_id'][user_id] = {content_id}
            user_dict['lecture_in_2'][user_id] += 1
            user_dict['lag_timestamp'][user_id].append(timestamp)
            if len(user_dict['lag_timestamp'][user_id])>3:
                user_dict['lag_timestamp'][user_id] = user_dict['lag_timestamp'][user_id][-3:]
            seen_users.add(user_id)
        else:
            user_dict['content_id'][user_id].add(content_id)
        
        part_dict['part_cumsum'][(user_id,quest_dict['part'][content_id])] += answered_correctly
        part_dict['part_cumcount'][(user_id,quest_dict['part'][content_id])] += 1
        
        attempts_dict[user_id][content_id] = timestamp//(3600*24)
        
        user_dict['user_punish'][user_id] += (answered_correctly - qn_right_ans)
        user_dict['user_qn_solved'][user_id] += 1    
        user_dict['user_right_ans'][user_id] += answered_correctly
    
        quest_dict['qn_count'][content_id] += 1
        quest_dict['qn_right_ans'][content_id] += answered_correctly
        
        qn_right_dict[int(user_id)][int(content_id)] =  qn_right_dict[int(user_id)][int(content_id)] or answered_correctly

#         attempts_dict[user_id][content_id] = min(attempts_dict[user_id][content_id]+1,4)

In [12]:
def update_from_new(new_data):
    new_data = new_data.loc[new_data.content_type_id == False,['user_id','prior_question_elapsed_time','prior_question_had_explanation','timestamp']]
    new_data = new_data.dropna(subset=['prior_question_elapsed_time','prior_question_had_explanation'])
    new_data = new_data.drop_duplicates(subset='user_id',keep='first')
    
    new_data['prior_question_elapsed_time'] = np.rint(new_data['prior_question_elapsed_time']/1000).astype('uint16')
    new_data['prior_question_had_explanation'] = new_data['prior_question_had_explanation'].astype('uint8')

    for user_id, elapsed_time, expln_check, timestamp in new_data[['user_id','prior_question_elapsed_time','prior_question_had_explanation','timestamp']].values:
        user_dict['ans_var_15_day'][user_id] = user_dict['ans_var_15_day'].get(user_id,[])+[timestamp/(1000*60*60*24)]
        if len(user_dict['ans_var_15_day'][user_id]) > 15:
            user_dict['ans_var_15_day'][user_id].pop(0)
        
        for content_id in user_dict['content_id'].get(user_id,set()):
            user_dict['user_elapsed_time'][user_id] += elapsed_time
            user_dict['user_seen_expln'][user_id] += expln_check
            
            quest_dict['qn_median_elapsed_time'][content_id] = quest_dict['qn_median_elapsed_time'].get(content_id,na_fill['qn_median_elapsed_time']) + elapsed_time
            quest_dict['qn_expln_check'][content_id] = quest_dict['qn_expln_check'].get(content_id,na_fill['qn_expln_check']) + expln_check

            med_time = quest_dict['qn_median_elapsed_time'][content_id]/quest_dict['qn_count'].get(content_id,1)
            
#             time_perf = 1
#             if med_time != 0:
            time_perf = elapsed_time/med_time
            
            time_perf = min(time_perf,2)
            time_perf = max(time_perf,0.5)
            time_perf = np.log(time_perf)

            user_dict['user_time_performance'][user_id] += time_perf

            

In [13]:
# user_dict['content_id'] = {i:set() for i in user_dict['content_id'].keys()}

In [20]:
# example = pd.DataFrame({'row_id':[0,1,2,3,4,5],
#                        'group_num':[0,0,0,0,0,0],
#                        'timestamp':[0,10000,10000,25000,25000,50000],
#                        'user_id':[0,0,0,0,0,0],
#                        'content_id':[0,1,2,3,4,0],
#                        'content_type_id':[False,True,True,False,False,False],
#                        'task_container_id':[0,1,1,2,2,3],
#                        'prior_question_elapsed_time':[np.nan,10000,10000,5000,5000,3000],
#                        'prior_question_had_explanation':[np.nan,False,False,True,True,False],
#                        'prior_group_answers_correct':['[]','[1]','','[1,1]','','[0,1]'],
#                        'prior_group_responses':[[],[],[],[],[],[]]})
# separations = example[example.prior_group_answers_correct != ""].index.to_list()
# separations.append(len(example))

# example

,row_id,group_num,timestamp,user_id,content_id,content_type_id,task_container_id,prior_question_elapsed_time,prior_question_had_explanation,prior_group_answers_correct,prior_group_responses
0,0,0,0,0,0,False,0,NaN,NaN,[],[]
1,1,0,10000,0,1,True,1,10000.0,False,[1],[]
2,2,0,10000,0,2,True,1,10000.0,False,,[]
3,3,0,25000,0,3,False,2,5000.0,True,"[1,1]",[]
4,4,0,25000,0,4,False,2,5000.0,True,,[]
5,5,0,50000,0,0,False,3,3000.0,False,"[0,1]",[]


In [21]:
# example = dt.fread("../input/riiid-test-answer-prediction/example_test.csv").to_pandas()
# separations = example[example.prior_group_answers_correct != ""].index.to_list()
# separations.append(len(example))
# separations

In [22]:
# example.loc[1:3,'content_type_id'] = True
# example.loc[1:3,['prior_question_elapsed_time','prior_question_had_explanation']] = np.nan
# example.loc[1:3,'content_id'] = [30000,30001,30002]

In [23]:
# %%time

# res = []
# ans = []
# old_data = None
# for i,j in zip(separations[:4],separations[1:]):
#     new_data = example.iloc[i:j]
        
#     if old_data is not None:
#         old_data['answered_correctly'] = eval(new_data['prior_group_answers_correct'].iloc[0])
#         update_from_old(old_data)
#         update_from_new(new_data)
        
#     new_data = preprocess(new_data)

#     old_data = new_data.copy()
#     new_data['answered_correctly'] = model.predict(new_data[train_columns])
    
#     ans.append(new_data.loc[new_data['content_type_id'] == 0, ['row_id', 'answered_correctly']])
#     res.append(new_data)
# res = pd.concat(res)
# ans = pd.concat(ans)

CPU times: user 873 ms, sys: 21.1 ms, total: 894 ms
Wall time: 261 ms


In [18]:
import riiideducation
import pandas as pd
import numpy as np

env = riiideducation.make_env()
iter_test = env.iter_test()

In [19]:
%%time

old_data = None
for (new_data, sample_prediction_df) in iter_test:
    if old_data is not None:
        old_data['answered_correctly'] = eval(new_data['prior_group_answers_correct'].iloc[0])
        update_from_old(old_data)
        update_from_new(new_data)

    new_data = preprocess(new_data)
    old_data = new_data.copy()
    new_data['answered_correctly'] = model.predict(new_data[train_columns])
    env.predict(new_data.loc[new_data['content_type_id'] == 0, ['row_id', 'answered_correctly']])

CPU times: user 1.7 s, sys: 93.6 ms, total: 1.79 s
Wall time: 956 ms
